# lab 11-3_Class , Layers, Ensemble using pytorch
1. model Class를 더욱 체계적이고 견고하게 짜봅시다.<br>
2. Layer를 Sequential 로 더 이쁘게 구축하기<br>
3. 도출된 여러개의 model을 짬뽕하여 하나의 model로 만드는 Ensemble 이용

In [103]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch import nn, optim
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init

In [104]:
#MNIST data
mnist_trainset = datasets.MNIST(root = './data',train =True,transform=transforms.ToTensor(),download=True)
mnist_testset = datasets.MNIST(root = './data', train = False, transform=transforms.ToTensor(),download=True)

In [105]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
keep_prob = 0.7

# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=mnist_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=mnist_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [106]:
class Model(nn.Module):
    def __init__(self,name):
        super(Model, self).__init__()
        self._build_net()
        self.name = name
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    # Instance 생성 즉시 _build_net() 실행
    def _build_net(self):
        # Layer #1
        self.layer1 = nn.Sequential(nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding =1),
                                    nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride =2),
                                    nn.Dropout(p = 1- keep_prob)
                                   )
        # Layer #2
        self.layer2 = nn.Sequential(nn.Conv2d(32,64,kernel_size = 3, stride = 1, padding =1),
                                    nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride =2),
                                    nn.Dropout(p = 1- keep_prob)
                                   )        
        # Layer #3
        self.layer3 = nn.Sequential(nn.Conv2d(64,128,kernel_size = 3, stride = 1, padding =1),
                                    nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride =2, padding = 1),
                                    nn.Dropout(p = 1- keep_prob)
                                   ) 
        # FC Layer #1
        self.fc1_ = nn.Linear(4*4*128, 625, bias = True)
        nn.init.xavier_uniform_(self.fc1_.weight)
        self.fc1 = nn.Sequential(self.fc1_,
                                 nn.ReLU(),
                                   nn.Dropout(p = 0.5))
        # FC Layer #2 # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625, 10, bias = True)
        nn.init.xavier_uniform_(self.fc2.weight)
   
        
    def forward(self, x):
        
        output = self.layer1(x)
        output = self.layer2(output)
        output = self.layer3(output)
        in_size = output.size(0)
        output = output.view(in_size,-1)
        output = self.fc1(output)
        output = self.fc2(output)
        return F.log_softmax(output,dim = 1)
        
    def predict(self, x_test):
        self.eval()
        
        return self.forward(x_test)
    
    def get_accuracy(self, x_test, y_test):
        y_pred = self.predict(x_test)
        return y_pred, self.criterion(y_pred,y_test).item()
    def train_data(self, x_data, y_data):
        self.train()
        self.optimizer.zero_grad()
        y_pred = self.forward(batch_xs)
        loss = self.criterion(y_pred,y_data)
        loss.backward()
        self.optimizer.step()
        return loss, self.optimizer

In [107]:
model = Model("CNN")

In [ ]:
for epoch in range(training_epochs):
    # Train
    avg_cost = 0
    total_batch = int(len(mnist_trainset) / batch_size)

    for idx, (batch_xs, batch_ys) in enumerate(train_loader):
        
        loss, optimizer = model.train_data(batch_xs,batch_ys)
        avg_cost += loss / total_batch
    print('\nTrain Epoch: {}\tLoss: {:.6f}'.format(epoch, avg_cost))
    
    # TEST
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output,acc = model.get_accuracy(data, target)
        test_loss += acc
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    
    # print eval
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0	Loss: 0.298433
Test set: Average loss: 0.0005, Accuracy: 9849/10000 (98%)


Train Epoch: 1	Loss: 0.085517
Test set: Average loss: 0.0003, Accuracy: 9892/10000 (98%)

